> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [ ]:
import pandas as pd
import requests
from io import StringIO

# URL do arquivo CSV no GitHub
url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"

# Baixar o conteúdo do arquivo
response = requests.get(url)
response.raise_for_status()  # Verifica se a requisição foi bem-sucedida

# Carregar os dados em um DataFrame
data = pd.read_csv(StringIO(response.text))

# Visualizar as primeiras linhas do DataFrame
print("Primeiras 5 linhas do DataFrame:")
print(data.head())

# Informações gerais sobre os dados
print("\nInformações sobre o DataFrame:")
print(data.info())

# Estatísticas descritivas
print("\nEstatísticas descritivas:")
print(data.describe())

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from enum import Enum as PyEnum
import pandas as pd

# Definindo Enums para campos categóricos
class SexEnum(PyEnum):
    MALE = 'M'
    FEMALE = 'F'

class DesignationEnum(PyEnum):
    ANALYST = 'Analyst'
    SENIOR_ANALYST = 'Senior Analyst'
    DATA_SCIENTIST = 'Data Scientist'
    DATA_ENGINEER = 'Data Engineer'
    MANAGER = 'Manager'

class UnitEnum(PyEnum):
    IT = 'IT'
    FINANCE = 'Finance'
    MARKETING = 'Marketing'
    HR = 'HR'
    OPERATIONS = 'Operations'

# Configuração do SQLAlchemy
Base = declarative_base()

class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    sex = Column(Enum(SexEnum), nullable=False)
    doj = Column(Date, nullable=False)  # Date of Joining
    current_date = Column(Date, nullable=False)
    designation = Column(Enum(DesignationEnum), nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnitEnum), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)  # Past Experience

# Criar engine e tabelas
engine = create_engine('sqlite:///salaries.db')
Base.metadata.create_all(engine)

# Criar sessão
Session = sessionmaker(bind=engine)
session = Session()

# Carregar dados do CSV
df = pd.read_csv('salaries.csv')

# Converter dados para o formato do ORM
for _, row in df.iterrows():
    employee = Employee(
        first_name=row['FIRST NAME'],
        last_name=row['LAST NAME'],
        sex=SexEnum(row['SEX']),
        doj=pd.to_datetime(row['DOJ']).date(),
        current_date=pd.to_datetime(row['CURRENT DATE']).date(),
        designation=DesignationEnum(row['DESIGNATION']),
        age=row['AGE'],
        salary=row['SALARY'],
        unit=UnitEnum(row['UNIT']),
        leaves_used=row['LEAVES USED'],
        leaves_remaining=row['LEAVES REMAINING'],
        ratings=row['RATINGS'],
        past_exp=row['PAST EXP']
    )
    session.add(employee)

# Commit para salvar no banco
session.commit()
session.close()

#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [ ]:
from sqlalchemy import create_engine

# Criar uma conexão com o banco de dados SQLite chamado 'salarios.db'
engine = create_engine('sqlite:///salarios.db')

# Testar a conexão
try:
    with engine.connect() as connection:
        print("✅ Conexão com o banco de dados 'salarios.db' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ Erro ao conectar ao banco de dados: {e}")

#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, Enum
from sqlalchemy.ext.declarative import declarative_base
from enum import Enum as PyEnum
from datetime import datetime

# 1. Definir Enums para os campos categóricos
class Sex(PyEnum):
    M = 'M'
    F = 'F'

class Designation(PyEnum):
    ANALYST = 'Analyst'
    SENIOR_ANALYST = 'Senior Analyst'
    DATA_SCIENTIST = 'Data Scientist'
    DATA_ENGINEER = 'Data Engineer'
    MANAGER = 'Manager'
    # Adicione outros cargos conforme encontrados nos dados

class Unit(PyEnum):
    IT = 'IT'
    FINANCE = 'Finance'
    MARKETING = 'Marketing'
    HR = 'HR'
    OPERATIONS = 'Operations'
    # Adicione outros departamentos conforme encontrados nos dados

# 2. Configurar a base do SQLAlchemy
Base = declarative_base()

# 3. Criar a classe Employee que mapeia para a tabela
class Employee(Base):
    __tablename__ = 'employees'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50), nullable=False)
    last_name = Column(String(50), nullable=False)
    sex = Column(Enum(Sex), nullable=False)
    doj = Column(Date, nullable=False)  # Date of Joining
    current_date = Column(Date, nullable=False)
    designation = Column(Enum(Designation), nullable=False)
    age = Column(Integer, nullable=False)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(Unit), nullable=False)
    leaves_used = Column(Integer, nullable=False)
    leaves_remaining = Column(Integer, nullable=False)
    ratings = Column(Float, nullable=False)
    past_exp = Column(Float, nullable=False)  # Past Experience

    def __repr__(self):
        return f"<Employee(id={self.id}, name='{self.first_name} {self.last_name}')>"

# 4. Criar o banco de dados e as tabelas
engine = create_engine('sqlite:///salarios.db')
Base.metadata.create_all(engine)

print("Tabelas criadas com sucesso no banco 'salarios.db'")

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

# 1. Baixar e carregar o arquivo CSV
url = "https://raw.githubusercontent.com/camilalaranjeira/python-intermediario/main/salaries.csv"
df = pd.read_csv(url)

# 2. Converter as colunas de data para o formato datetime
df['DOJ'] = pd.to_datetime(df['DOJ'], format='%m/%d/%Y')
df['CURRENT DATE'] = pd.to_datetime(df['CURRENT DATE'], format='%m/%d/%Y')

# 3. Mostrar uma prévia dos dados
print("Primeiras 5 linhas do DataFrame:")
print(df.head())
print("\nInformações sobre o DataFrame:")
print(df.info())

# 4. Conectar ao banco de dados SQLite
engine = create_engine('sqlite:///salarios.db')

# 5. Popular o banco de dados usando to_sql
df.to_sql(
    name='employees',          # Nome da tabela
    con=engine,                # Conexão com o banco
    if_exists='append',        # Adicionar aos dados existentes
    index=False,               # Não incluir o índice do DataFrame
    dtype={
        'FIRST NAME': String(50),
        'LAST NAME': String(50),
        'SEX': Enum('M', 'F', name='sex'),
        'DOJ': Date,
        'CURRENT DATE': Date,
        'DESIGNATION': String(50),
        'AGE': Integer,
        'SALARY': Float,
        'UNIT': String(50),
        'LEAVES USED': Integer,
        'LEAVES REMAINING': Integer,
        'RATINGS': Float,
        'PAST EXP': Float
    }
)

print("\nDados importados com sucesso para o banco 'salarios.db'!")

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [ ]:
from sqlalchemy import text

# Query SQL pura
query_sql = """
SELECT 
    DESIGNATION,
    MIN(SALARY/12) as min_salary_monthly,
    MAX(SALARY/12) as max_salary_monthly,
    AVG(SALARY/12) as avg_salary_monthly
FROM 
    employees
GROUP BY 
    DESIGNATION
"""

# Executando com engine.connect()
with engine.connect() as conn:
    result = conn.execute(text(query_sql))
    print("\nResultado usando engine.connect():")
    for row in result:
        print(row)

In [ ]:
import pandas as pd

# Executando com pandas
df_result = pd.read_sql_query(query_sql, engine)

print("\nResultado usando pandas.read_sql_query():")
print(df_result)

In [ ]:
from sqlalchemy import select, func
from sqlalchemy.orm import Session

# Criando a query com SQLAlchemy ORM
stmt = select(
    Employee.designation,
    func.min(Employee.salary/12).label('min_salary_monthly'),
    func.max(Employee.salary/12).label('max_salary_monthly'),
    func.avg(Employee.salary/12).label('avg_salary_monthly')
).group_by(Employee.designation)

# Executando com Session
with Session(engine) as session:
    result = session.execute(stmt)
    print("\nResultado usando SQLAlchemy ORM:")
    for row in result:
        print(row)